Training code is based on: https://huggingface.co/docs/transformers/tasks/multiple_choice.
Evaluation is based on: https://medium.com/nlplanet/bert-finetuning-with-hugging-face-and-training-visualizations-with-tensorboard-46368a57fc97

In [ ]:
import pandas as pd

# Load data
dir = "common-sense"

train_file = os.path.join(dir, "train_data.csv")
train_labels_file = os.path.join(dir, "train_answers.csv")

df_sentences = pd.read_csv(train_file)
df_labels = pd.read_csv(train_labels_file)

In [ ]:
# Combine sentences and labels dataframes
df = pd.merge(df_sentences, df_labels, on="id").drop(["id"], axis=1)
df = df.rename(columns={"FalseSent": "sent", "answer": "label"})

In [ ]:
def label(x):
    """Convert options to (integer) labels"""
    if x == 'A':
        return 0
    elif x == 'B':
        return 1
    else:
        return 2
    
df['label'] = df['label'].apply(label)

In [ ]:
df.head()

In [ ]:
from datasets import Dataset, dataset_dict, load_metric

# Convert to huggingface dataset and split in train/test
data = Dataset.from_pandas(df).train_test_split(test_size=0.2)
data = data.class_encode_column("label")

In [ ]:
from transformers import AutoTokenizer

options = ['OptionA', 'OptionB', 'OptionC']

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples: Dataset) -> dataset_dict:
    """ Perform preprocessing of the input.
        # Arguments
        Dataset: Huggingface Dataset 
        # Output
        Huggingface dataset_dict with the tokenized examples 
        with corresponding input_ids, attention_mask, and labels.
    """
    # Make copies of sent field
    copied_sent = [[i] * 3 for i in examples["sent"]]

    # Combine FalseSent with 3 possible options
    options_sents = [
        [f"{header} {examples[option][i]}" for option in options] for i, header in enumerate(examples["sent"])
    ]

    # Flatten lists
    copied_sent = sum(copied_sent, [])
    options_sents = sum(options_sents, [])

    # Tokenize flattened lists
    tokenized_examples = tokenizer(copied_sent, options_sents, truncation=True)

    # Unflatten 
    return {k: [v[i : i + 3] for i in range(0, len(v), 3)] for k, v in tokenized_examples.items()}

# Apply preprocess function on entire dataset
tokenized = data.map(preprocess_function, batched=True)

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    Flattens all model inputs, apply padding, unflatten results.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

# Load BERT model
MODEL_NAME = "bert-base-uncased"
CHECKPOINT = "checkpoint-500/"

OUTPUT_DIR = f"results/{MODEL_NAME}-{CHECKPOINT}"

# Load model either new or from last checkpoint
try:
    model = AutoModelForMultipleChoice.from_pretrained(f"results/{CHECKPOINT}")
except:
    model = AutoModelForMultipleChoice.from_pretrained(MODEL_NAME)

In [ ]:
training_args = TrainingArguments(
    output_dir = OUTPUT_DIR,
    evaluation_strategy = "epoch",
    learning_rate = 5e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 3, # Default = 3
    weight_decay = 0.01,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized["train"],
    eval_dataset = tokenized["test"],
    tokenizer = tokenizer,
    data_collator = DataCollatorForMultipleChoice(tokenizer = tokenizer),
    )

trainer.train()

In [ ]:
import numpy as np

metric = load_metric("accuracy")

test_predictions = trainer.predict(tokenized["test"])

# For each prediction, create the label with argmax
test_predictions_argmax = np.argmax(test_predictions[0], axis=1)
# Retrieve reference labels from test set
test_references = np.array(tokenized["test"]["label"])
# Compute accuracy
metric.compute(predictions=test_predictions_argmax, references=test_references)